In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://hl-codon-09-03.ebi.ac.uk:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /nfs/research/birney/users/shimin/ukbb/wes/200k/UKBBurden/scripts/hail-20210508-0923-0.2.61-3c86d3ba497a.log


In [2]:
sid = 'ukb23156_c5_b28_v1'
inVCF = f'../../pvcfQC/1_norm/{sid}.norm.vcf.gz'
outVCF = f'../../pvcfQC/3_hail_qc/{sid}.qc.norm.vcf.bgz'
print(inVCF, outVCF)

../../pvcfQC/1_norm/ukb23156_c5_b28_v1.norm.vcf.gz ../../pvcfQC/3_hail_qc/ukb23156_c5_b28_v1.qc.norm.vcf.bgz


In [3]:
mt = hl.import_vcf(inVCF, min_partitions=4, reference_genome='GRCh38', array_elements_required=False, force_bgz=True)
n = mt.count()
n_var0 = n[0]
print(n)
# Genotype quality control
## Set filter condition for AB
mt = mt.annotate_entries(AB = (mt.AD[1] / hl.sum(mt.AD)))
filter_condition_ab = ((mt.GT.is_hom_ref() & (mt.AB <= 0.1)) | (mt.GT.is_het() & (mt.AB >= 0.2) & (mt.AB <= 0.8)) | (mt.GT.is_hom_var() & (mt.AB >= 0.9)))
filter_condition = (mt.GQ >= 20) & (mt.DP >= 10) & filter_condition_ab
fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition))
print(f'>{fraction_filtered * 100:.2f}% entries filtered out of downstream analysis.')
mt = mt.filter_entries(filter_condition)
## Add frac of samples passing AB threshold per row
mt = mt.annotate_rows(AB_count = hl.agg.count_where((mt.GT.is_het() & (mt.AB >= 0.2) & (mt.AB <= 0.8))))

# Variant quality control
mt = hl.variant_qc(mt)
mt = mt.filter_rows((mt.AB_count > 0) & (mt.variant_qc.call_rate >= 0.9) & (mt.variant_qc.p_value_hwe >= 1e-15))

n = mt.count()
n_var1 = n[0]
print(f'variantsPASS,{outVCF},{n_var1},{n_var0}')

mt = mt.annotate_rows(info = mt.info.annotate(AF=mt.variant_qc.AF[1]))
hl.export_vcf(mt, outVCF)

2021-05-08 09:23:57 Hail: INFO: Coerced almost-sorted dataset


(17339, 200643)


2021-05-08 09:24:27 Hail: INFO: Coerced almost-sorted dataset


>0.91% entries filtered out of downstream analysis.


2021-05-08 09:26:03 Hail: INFO: Coerced almost-sorted dataset


variantsPASS,../../pvcfQC/3_hail_qc/ukb23156_c5_b28_v1.qc.norm.vcf.bgz,16901,17339


2021-05-08 09:27:25 Hail: WARN: export_vcf: ignored the following fields:
    'AB_count' (row)
    'variant_qc' (row)
2021-05-08 09:27:38 Hail: INFO: Coerced almost-sorted dataset
2021-05-08 09:38:02 Hail: INFO: merging 71 files totalling 9.8G...
2021-05-08 09:38:23 Hail: INFO: while writing:
    ../../pvcfQC/3_hail_qc/ukb23156_c5_b28_v1.qc.norm.vcf.bgz
  merge time: 21.456s
